## Melon 주간 차트 크롤링 - beautifulsoup 이용

In [1]:
import pandas as pd
import requests

In [6]:
# 한빛 아카데미의 경우 - 정보가 쭉 나온다. 
url = "https://www.hanbit.co.kr/academy/"
req = requests.get(url)
html = req.text
# html

In [7]:
# 멜론의 경우 - 막아놓아서 정보가 따로 나오지 않는다. 
url = "https://www.melon.com/chart/week/index.htm"
req = requests.get(url)
html = req.text
html

''

Console창에서 
navigator.userAgent를 한 뒤 나오는 값을 header로 줘야만 정보를 내어준다.

In [9]:
# 방법
header = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
url = "https://www.melon.com/chart/week/index.htm"
req = requests.get(url,headers = header)
html = req.text
# html

html파일 열기

In [11]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

* selector 방식으로 찾기

In [ ]:
#lst50 > td:nth-child(6) > div > div > div.ellipsis.rank01 > span > a

* Copy Xpath 방식으로 찾기

In [ ]:

//*[@id="lst50"]/td[6]/div/div/div[1]/span/a

* beautiful soup - id 방식으로 찾기

In [12]:
trs = soup.select('#lst50')         #id
len(trs)                            # 50개가 trs에 다 들어가 있는 상태

50

### 하나 가지고 test

In [13]:
# 데이터 한 즐 확인
tr = trs[0]
# tr

<tr class="lst50" data-song-no="33507137" id="lst50">
<td><div class="wrap t_right"><input class="input_check" name="input_check" title="Butter 곡 선택" type="checkbox" value="33507137"/></div></td>
<td><div class="wrap t_center"><span class="rank">1</span><span class="none">위</span></div></td>
<!-- 차트순위 추가 -->
<td><div class="wrap">
<span class="rank_wrap" title="14단계 상승">
<span class="bullet_icons rank_up"><span class="none">단계 상승</span></span>
<span class="up">14</span>
</span>
</div></td>
<td><div class="wrap">
<a class="image_typeAll" href="javascript:melon.link.goAlbumDetail('10612483');" title="Butter">
<img alt="Butter - 페이지 이동" height="60" onerror="WEBPOCIMG.defaultAlbumImg(this);" src="https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize" width="60">
<span class="bg_album_frame"></span>
</img></a>
</div></td>
<td><div class="wrap">
<a class="btn button_icons type03 song_info" href="javascript:melon.link.goSon

순위 가져오기

In [14]:
rank = tr.select_one('.rank')                   # class = "rank"인 것 찾기
rank

<span class="rank">1</span>

In [16]:
rank= tr.select_one(".rank").string             # 원하는 것을 얻기 위해서는 한단계 더 가야한다. get_text() 쓰기도 함.
rank = int(rank)
rank

1

곡 제목 가져오기  
참고) 클래스 하나 밑으로 갈 때 마다 중간에 . 을 추가한다.  
* soup.find : 해당 조건에 맞는 가장 첫번째 값을 가져온다.  
* soup.find_all : 해당 조건에 맞는 모든 태그들을 가져온다.  

In [19]:
title = tr.select_one('.ellipsis.rank01')
title = title.find('a').get_text()           # 'a'태그를 가져와서 문자를 추출하기
title

'Butter'

가수명 가져오기

In [23]:
#곡 제목과 같은 방식으로 진행
artist = tr.select_one('.ellipsis.rank02')
artist = artist.find('a').get_text()
artist

'방탄소년단'

앨범명 가져오기

In [24]:
album = tr.select_one('.ellipsis.rank03')
album = album.find('a').string
album

'Butter'

### 50곡에 대해서 데이터프레임 만들기

In [25]:
rank_list, title_list, artist_list, album_list = [],[],[],[]

for tr in trs:
    rank= tr.select_one(".rank").string
    rank = int(rank)

    title = tr.select_one('.ellipsis.rank01')
    title = title.find('a').get_text()

    artist = tr.select_one('.ellipsis.rank02')
    artist = artist.find('a').get_text()

    album = tr.select_one('.ellipsis.rank03')
    album = album.find('a').string

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)


df = pd.DataFrame({'순위' : rank_list, '곡명' : title_list, '가수' : artist_list, '앨범' : album_list})
df.head()

,순위,곡명,가수,앨범
0,1,Butter,방탄소년단,Butter
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL
2,3,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice
3,4,헤픈 우연,헤이즈 (Heize),HAPPEN
4,5,롤린 (Rollin'),브레이브걸스,Rollin'


* 이미지 속성값 가져오기

In [28]:
image = trs[0].select_one(".image_typeAll").find('img')
image = image['src']
image

'https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize'

In [35]:
image_list = []
for tr in trs:
    image = tr.select_one(".image_typeAll").find('img')
    image = image['src']
    image_list.append(image)

df['이미지'] = image_list

100까지 가져오기

In [29]:
trs100 = soup.select('#lst100')         
len(trs100)  

50

In [30]:
# trs100[0]

<tr class="lst100" data-song-no="33229299" id="lst100" style="">
<td><div class="wrap t_right"><input class="input_check" name="input_check" title="이 밤을 빌려 말해요 (바른연애 길잡이 X 10CM) 곡 선택" type="checkbox" value="33229299"/></div></td>
<td><div class="wrap t_center"><span class="rank">51</span><span class="none">위</span></div></td>
<!-- 차트순위 추가 -->
<td><div class="wrap">
<span class="rank_wrap" title="8단계 하락">
<span class="bullet_icons rank_down"><span class="none">단계 하락</span></span>
<span class="down">8</span>
</span>
</div></td>
<td><div class="wrap">
<a class="image_typeAll" href="javascript:melon.link.goAlbumDetail('10552055');" title="이 밤을 빌려 말해요 (바른연애 길잡이 X 10CM)">
<img alt="이 밤을 빌려 말해요 (바른연애 길잡이 X 10CM) - 페이지 이동" height="60" onerror="WEBPOCIMG.defaultAlbumImg(this);" src="https://cdnimg.melon.co.kr/cm2/album/images/105/52/055/10552055_20210122153304_500.jpg/melon/resize/120/quality/80/optimize" width="60"/>
<span class="bg_album_frame"></span>
</a>
</div></td>
<td><div class="wrap">


In [31]:
for tr in trs100:
    rank= tr.select_one(".rank").string
    rank = int(rank)

    title = tr.select_one('.ellipsis.rank01')
    title = title.find('a').get_text()

    artist = tr.select_one('.ellipsis.rank02')
    artist = artist.find('a').get_text()

    album = tr.select_one('.ellipsis.rank03')
    album = album.find('a').string

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)


df100 = pd.DataFrame({'순위' : rank_list, '곡명' : title_list, '가수' : artist_list, '앨범' : album_list})
df100.head()

,순위,곡명,가수,앨범
0,1,Butter,방탄소년단,Butter
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL
2,3,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice
3,4,헤픈 우연,헤이즈 (Heize),HAPPEN
4,5,롤린 (Rollin'),브레이브걸스,Rollin'


In [39]:
for tr in trs100:
    image = tr.select_one(".image_typeAll").find('img')
    image = image['src']
    image_list.append(image)

df100['이미지'] = image_list

In [40]:
df100.tail()

,순위,곡명,가수,앨범,이미지
95,96,혼술하고 싶은 밤,벤,혼술하고 싶은 밤,https://cdnimg.melon.co.kr/cm2/album/images/10...
96,97,죄와벌,SG 워너비,살다가,https://cdnimg.melon.co.kr/cm/album/images/000...
97,98,되풀이,임창정,펜트하우스2 OST Part.1,https://cdnimg.melon.co.kr/cm2/album/images/10...
98,99,고래 (Dive Into You),NCT DREAM,맛 (Hot Sauce) - The 1st Album,https://cdnimg.melon.co.kr/cm2/album/images/10...
99,100,돌림노래 (Feat. DEAN),아이유,IU 5th Album 'LILAC',https://cdnimg.melon.co.kr/cm2/album/images/10...


조금 더 간단하기

In [ ]:
trs = soup.select('#lst50')
trs100 = soup.select('#lst100')
trs.extend(trs100)
len(trs)

In [ ]:
rank_list, title_list, artist_list, album_list,image_list = [],[],[],[],[]

for tr in trs:
    rank= tr.select_one(".rank").string
    rank = int(rank)

    title = tr.select_one('.ellipsis.rank01')
    title = title.find('a').get_text()

    artist = tr.select_one('.ellipsis.rank02')
    artist = artist.find('a').get_text()

    album = tr.select_one('.ellipsis.rank03')
    album = album.find('a').string

    image = tr.select_one(".image_typeAll").find('img')
    image = image['src']

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)
    image_list.append(image)


df = pd.DataFrame({'순위' : rank_list, '곡명' : title_list, '가수' : artist_list, '앨범' : album_list, '이미지' : image_list})
df.head()

* 파일에 저장하기

In [47]:
# 기간 추출
period = soup.select_one(".yyyymmdd").get_text().strip()
period

'2021.05.24 ~ 2021.05.30'

In [50]:
filename = f'melon {period}.csv'
df100.to_csv(filename,index=False,sep=",",encoding='utf-8')     # CSV로 저장완료